# Changing an environment registered in Sinergym

As discussed above, Sinergym has a list of available environments that we can call with `gym.make(<environment_id>)` as long as the sinergym package is imported into the python script.

In [9]:
import gym
import numpy as np

import sinergym
env = gym.make('Eplus-5Zone-hot-continuous-stochastic-v1')

[2022-10-03 11:06:11,813] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:06:11,813] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:06:11,814] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:06:11,814] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:06:11,816] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:06:11,816] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connecti

These environment IDs have a number of components defined, not only the building design (*IDF*), but also the reward function, the action and observation spaces, the defining variables, etc.

If you want a new environment, you can define it from scratch in our [environment list](https://github.com/ugr-sail/sinergym/blob/main/sinergym/__init__.py) and run it locally.

Another option (recommended) is to start from one of our environments and change the components you want. You can combine components you want to change obviously.

The way to do that is to add in the `gym.make(<environment_id>)` those parameters of the constructor of the *Sinergym* environment we want to change. Let's see what things we can change starting from any environment:

## Adding a new reward

As mentioned above, simply add the appropriate parameters to `gym.make()` after specifying the environment ID.

In [10]:
from sinergym.utils.rewards import LinearReward, ExpReward

env = gym.make('Eplus-5Zone-hot-continuous-v1', reward=ExpReward, reward_kwargs={
                                                                    'temperature_variable': 'Zone Air Temperature (SPACE1-1)',
                                                                    'energy_variable': 'Facility Total HVAC Electricity Demand Rate (Whole Building)',
                                                                    'range_comfort_winter': (20.0, 23.5),
                                                                    'range_comfort_summer': (23.0, 26.0),
                                                                    'energy_weight': 0.1})

[2022-10-03 11:06:15,333] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:06:15,333] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:06:15,334] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:06:15,334] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:06:15,336] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:06:15,336] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:06:15,337] EPLUS_ENV_5Zone-hot-continuous-v1_Ma

You have to specify the **reward** class you are going to use in the environment. A reward function class has several parameters that can be specified by user like temperature variables, weights, etc. Depending on the reward function you are using. In order to be able to define it, we have **reward_kwargs** parameter. See [reward documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/rewards.html) for more information about reward classes and how to create a new one. 

## Adding other new components to the environment

In the same way that we can change the default reward function, as we have done in the second example, 
it is possible to substitute other default values of the environment ID. 

You can change the weather file, the number of timesteps an action repeats (default 1), 
the last n episodes you want to be stored in the Sinergym output folder (default 10), 
the name of the environment or the variability in stochastic environments:

In [11]:
env = gym.make('Eplus-5Zone-cool-continuous-stochastic-v1', 
                weather_file='ESP_Granada.084190_SWEC.epw',
                weather_variability=(1.0,0.0,0.001),
                env_name='new_env_name',
                act_repeat=4,
                max_ep_data_store_num = 20)

[2022-10-03 11:06:19,050] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:06:19,050] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:06:19,052] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:06:19,052] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:06:19,054] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:06:19,054] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:06:19,056] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Setting up extra configuration in building model if exists

## Changing observation and action spaces

By default, the IDs of the predefined environments in *Sinergym* already have a space of actions and observations set.

However, it can be overwritten by a new definition of them. On the one hand, we will have to define the name of the 
**variables**, and on the other hand, the definition of the **spaces** (and an **action mapping** if it is a discrete environment).

In [13]:
import gym
import numpy as np

import sinergym

new_observation_variables=[
    'Site Outdoor Air Drybulb Temperature(Environment)',
    'Site Outdoor Air Relative Humidity(Environment)',
    'Site Wind Speed(Environment)',
    'Zone Thermal Comfort Fanger Model PPD(East Zone PEOPLE)',
    'Zone People Occupant Count(East Zone)',
    'People Air Temperature(East Zone PEOPLE)',
    'Facility Total HVAC Electricity Demand Rate(Whole Building)'
]

new_action_variables = [
    'Heating_Setpoint_RL',
    'Cooling_Setpoint_RL',
]

new_observation_space = gym.spaces.Box(
    low=-5e6,
    high=5e6,
    shape=(len(new_observation_variables) + 4,),
    dtype=np.float32)

new_action_mapping = {
    0: (15, 30),
    1: (16, 29),
    2: (17, 28),
    3: (18, 27),
    4: (19, 26),
    5: (20, 25),
    6: (21, 24),
    7: (22, 23),
    8: (22, 22),
    9: (21, 21)
}

new_action_space = gym.spaces.Discrete(10)

env = gym.make('Eplus-datacenter-cool-discrete-stochastic-v1', 
                observation_variables=new_observation_variables,
                observation_space=new_observation_space,
                action_variables=new_action_variables,
                action_mapping=new_action_mapping,
                action_space=new_action_space
            )


for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env.action_space.sample()
        obs, reward, done, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

[2022-10-03 11:07:47,922] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:07:47,922] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:07:47,922] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:07:47,922] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:07:47,924] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:07:47,924] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10

Reward:  -0.2153129771240179 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 4306.259542480358, 'total_power_no_units': -0.4306259542480358, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [], 'out_temperature': 1.044000531676663, 'action_': [19, 26]}
Reward:  -660.6448328594148 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 4157.813830820063, 'total_power_no_units': -0.41578138308200635, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [], 'out_temperature': 8.755699132506065, 'action_': [18, 27]}
Reward:  -1244.0392548144782 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'total_power': 4725.134398205902, 'total_power_no_units': -0.4725134398205902, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [], 'out_temperature': 3.388003680767862, 'action_': [20, 25]}
Reward:  -1847.362623921392 {'timestep

[2022-10-03 11:08:18,405] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-10-03 11:08:18,405] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-10-03 11:08:18,405] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-10-03 11:08:18,405] EPLUS_ENV_datacenter-cool-discrete-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


In case the definition has some inconsistency, such as the *IDF* has not been 
adapted to the new actions, the spaces do not fit with the variables, the 
observation variables do not exist, etc. *Sinergym* will display an error.

**Action variables** defined must exists in IDF model like external interface.
Then, you have two possibilities: 

- First, you can modify IDF file directly, but it is necessary to update several 
  components depending in order to connect them to the external interface 
  (requires knowledge of the building and EnergyPlus).
- Second, you can use the **action definition** functionality of *Sinergym* (recommended).
  This functionality will modify the IDF file before starting the simulation automatically 
  based on a simple definition of the components to be replaced by an external control.
  See "updating action definition" example to learn how to use it.


## Getting information about building model with Sinergym

Sinergym can get the schedulers information automatically, in a Python dictionary and/or exporting that information in a excel file.
This information lets you know what things can be controlled by an external interface in the building.
The way to do it is very simple, just load the environment and call a method:

In [15]:
import gym
import sinergym

from pprint import pprint

env = gym.make('Eplus-demo-v1')
schedulers=env.get_schedulers()
pprint(schedulers)

[2022-10-03 11:08:48,685] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:08:48,685] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:08:48,686] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:08:48,686] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:08:48,688] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:08:48,688] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:08:48,689] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2022-10-03 11:08:48,689] EPLUS

{'activitysch': {'Type': 'any number'},
 'actschd': {'Object1': {'object_field_name': 'activity_level_schedule_name',
                         'object_name': 'space1-1 people 1',
                         'object_type': 'People'},
             'Object2': {'object_field_name': 'activity_level_schedule_name',
                         'object_name': 'space2-1 people 1',
                         'object_type': 'People'},
             'Object3': {'object_field_name': 'activity_level_schedule_name',
                         'object_name': 'space3-1 people 1',
                         'object_type': 'People'},
             'Object4': {'object_field_name': 'activity_level_schedule_name',
                         'object_name': 'space4-1 people 1',
                         'object_type': 'People'},
             'Object5': {'object_field_name': 'activity_level_schedule_name',
                         'object_name': 'space5-1 people 1',
                         'object_type': 'People'},
          

The way to export an excel file with such organized information is by specifying a path to the method:

In [17]:
actuators=env.get_schedulers(path='./example.xlsx')

It is possible to know the zones available in the environment too. You only have to do the next:

In [18]:
print(env.get_zones())

['plenum-1', 'space1-1', 'space2-1', 'space3-1', 'space4-1', 'space5-1']


## Updating the action definition of the environment

As we have explained in the action and observation space example, one of the problems that can arise when 
modifying the space of actions and observations is that the *IDF* is not adapted to the 
new space of actions established.

For this purpose, the *Sinergym* **action definition** is available. With a dictionary we can 
build a definition of what scheduler we want to be replaced in the building by the external control.

For this example, we are going to add lights control to the external interface for the agent.
Then, is not only necessary to update that **action definition**, the list of the action
variables and space is necessary to be updated too.

In [19]:
import gym
import numpy as np

new_action_variables=[
'Heating_Setpoint_RL',
'Cooling_Setpoint_RL',
'light_control'
]

new_action_space=gym.spaces.Box(
    low=np.array([15.0, 22.5, 0.0]),
    high=np.array([22.5, 30.0, 1.0]),
    shape=(3,),
    dtype=np.float32
)

new_action_definition={
    'Htg-SetP-Sch': {'name': 'Heating_Setpoint_RL', 'initial_value': 21},
    'Clg-SetP-Sch': {'name': 'Cooling_Setpoint_RL', 'initial_value': 25},
    'LIGHTS-1':{'name':'light_control','initial_value':0.2}

}

env = gym.make('Eplus-5Zone-cool-continuous-stochastic-v1',
                action_variables=new_action_variables,
                action_space=new_action_space,
                action_definition=new_action_definition
                )

for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env.action_space.sample()
        obs, reward, done, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
[2022-10-03 11:09:11,535] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:09:11,535] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:09:11,535] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:09:11,537] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:09:11,537] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:09:11,537] EPLUS_ENV

Reward:  -0.4534218472503313 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 2877.012363757218, 'total_power_no_units': -0.2877012363757218, 'comfort_penalty': -0.6191424581249407, 'abs_comfort': 0.6191424581249407, 'temperatures': [19.38085754187506], 'out_temperature': 2.365503840332209, 'action_': [17.785645, 29.71652, 0.22971705]}
Reward:  -1357.8271189188051 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 11100.46077251977, 'total_power_no_units': -1.110046077251977, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [21.21328190545486], 'out_temperature': 8.522913717556069, 'action_': [22.275938, 25.327274, 0.4951168]}
Reward:  -2677.3228805407884 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'total_power': 701.0042651760367, 'total_power_no_units': -0.07010042651760368, 'comfort_penalty': -0.9436218379550496, 'abs_com

[2022-10-03 11:09:23,032] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-10-03 11:09:23,032] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-10-03 11:09:23,032] EPLUS_ENV_5Zone-cool-continuous-stochastic-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


Previously, it is recommended to know what controllers are available in the specific building model, see the previous example of this notebook. For more information about the format of the action definition dictionaries, visit the section called [action definition](https://ugr-sail.github.io/sinergym/compilation/html/pages/environments.html#action-definition).

## Adding more extra configuration

You can even add a dictionary with extra parameters that update IDF with some new context concerned with simulation directly.

This new IDF version, which also adapts to the new weather you put in, is saved in the Sinergym output folder, leaving the original intact:

In [20]:
extra_conf={
    'timesteps_per_hour':6,
    'runperiod':(1,1,1991,2,1,1992),
}

env = gym.make('Eplus-datacenter-cool-continuous-stochastic-v1', 
                config_params=extra_conf
                )

[2022-10-03 11:10:20,132] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-10-03 11:10:20,133] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-10-03 11:10:20,136] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-10-03 11:10:20,141] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2022-10-03 11:10:20,143] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


For more information about extra configuration parameters, see our [Extra configuration documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/extra-configuration.html)